# ARGOS Single-Hotel Demo (Colab-ready)

This notebook demonstrates how to:

1. Install dependencies
2. Load the ARGOS hotel optimization package
3. Run the single-hotel ARGOS loop
4. Inspect basic metrics (occupancy, staff, fatigue, RevPAR)

You can upload the repository zip into Colab or clone from GitHub.


In [ ]:
# If running in Colab and you uploaded the zip to /content:
# !unzip -o argos_hotel_optimization_repo.zip -d /content/argos_repo

import os
os.chdir('/content/argos_repo') if os.path.exists('/content/argos_repo') else None


In [ ]:
# Install dependencies
!pip install -q -r requirements.txt


In [ ]:
# Add src to Python path
import sys, os
if 'src' not in os.listdir('.'):
    print("Warning: run this from the repo root (where `src/` lives).")
sys.path.append(os.path.abspath('src'))


In [ ]:
from argos.hotel_env import HotelEnvironment
from argos.lco import LexicographicTier, LexicographicTierStructure
from argos.cag import CAGFilter
from argos.argos_core import ARGOSOptimizer
import numpy as np


In [ ]:
def revenue_obj(x: np.ndarray) -> float:
    return - (x[0] * (50 + 100 * x[3]))

def revenue_grad(x: np.ndarray) -> np.ndarray:
    g = np.zeros_like(x)
    g[0] = - (50 + 100 * x[3])
    g[3] = - (100 * x[0])
    return g

def newton_step(x: np.ndarray) -> np.ndarray:
    H_inv = np.array([1e-3, 1.0, 1.0, 1e-3])
    return -H_inv * revenue_grad(x)

def contour_step(x: np.ndarray) -> np.ndarray:
    g = revenue_grad(x)
    norm = np.linalg.norm(g) + 1e-8
    return -0.1 * g / norm


In [ ]:
env = HotelEnvironment()
x = env.reset()

tier1 = LexicographicTier(
    name="Feasibility",
    objective=lambda x: 0.0,
    constraints=[lambda x: max(0.0, x[0] - 1.0)],  # occupancy <= 1.0
)
tiers = LexicographicTierStructure([tier1])
cag = CAGFilter(newton_step_fn=newton_step, contour_step_fn=contour_step)
argos = ARGOSOptimizer(tiers=tiers, cag=cag, step_size=0.05, highest_tier=1)

history = []
for t in range(100):
    x = argos.step(x, revenue_grad)
    x, metrics = env.step(x)
    metrics['epoch'] = t
    history.append(metrics)
history[:5]


In [ ]:
import pandas as pd
df = pd.DataFrame(history)
df.tail()


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(10, 6))
axes = axes.ravel()

axes[0].plot(df['epoch'], df['occupancy'])
axes[0].set_title('Occupancy')

axes[1].plot(df['epoch'], df['staff'])
axes[1].set_title('Staff level')

axes[2].plot(df['epoch'], df['fatigue'])
axes[2].set_title('Fatigue index')

axes[3].plot(df['epoch'], df['revpar'])
axes[3].set_title('RevPAR')

plt.tight_layout()
plt.show()
